In [ ]:
# Load in software packages
%matplotlib inline
from parcels import FieldSet, Field, VectorField, ParticleFile, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, AdvectionEE, plotTrajectoriesFile, Variable, BrownianMotion2D
from parcels import ErrorCode
from parcels import rng as random
import numpy as np
import math
from datetime import timedelta, datetime
from operator import attrgetter

In [ ]:
# Load in ocean model  Copernicus
filenames = {'U': "Copernicus/global-analysis-forecast-phy-001-024_1603882856077.nc",
             'V': "Copernicus/global-analysis-forecast-phy-001-024_1603882856077.nc"}
variables = {'U': 'uo',
             'V': 'vo'}
dimensions = {'lat': 'latitude',
              'lon': 'longitude',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions, allow_time_extrapolation=True)

In [ ]:
class SampleParticle(JITParticle):  # Define a new particle class that contains three extra variables
    age = Variable('age', dtype=np.float32, initial=0.) # initialise age
    distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                        initial=attrgetter('lon'))  # the previous longitude
    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                        initial=attrgetter('lat'))  # the previous latitude.
    #uwind = Variable('uwind', initial=fieldset.U)
    #vwind = Variable('vwind', initial=fieldset.V)
    kelpvelU = Variable('kelpvelU',initial=fieldset.U)
    kelpvelV = Variable('kelpvelV',initial=fieldset.V)

def SampleDistance(particle, fieldset, time): # Function measuring distance
    # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat
    
#def SampleAge(particle, fieldset, time):
#    particle.age = time

#def SampleWind(particle, fieldsetwind, time):
#    particle.uwind = fieldset.U[time, particle.depth, particle.lat, particle.lon]
#    particle.vwind = fieldset.V[time, particle.depth, particle.lat, particle.lon]

def SampleCurr(particle, fieldset, time):
    particle.currU = fieldset.U[time, particle.depth, particle.lat, particle.lon]
    particle.currV = fieldset.V[time, particle.depth, particle.lat, particle.lon]

def SampleKelpVel(particle, fieldset, time):
    particle.kelpvelU = fieldset.U[time, particle.depth, particle.lat, particle.lon]
    particle.kelpvelV = fieldset.V[time, particle.depth, particle.lat, particle.lon]
    
def DeleteParticle(particle, fieldset, time): # Function that deletes particle if it goes out of bounds to avoid OutOfBounds error
    particle.delete()

In [ ]:
#This is to add brownian motion to the particle/s

fset_currents = FieldSet.from_netcdf(filenames, variables, dimensions)
fset_currents.add_periodic_halo(zonal=True)
size2D = (fset_currents.U.grid.ydim, fset_currents.U.grid.xdim)

fieldset.add_field(Field('Kh_zonal', data=10 * np.ones(size2D),
                         lon=fset_currents.U.grid.lon, lat=fset_currents.U.grid.lat,
                         mesh='spherical', allow_time_extrapolation=True))
fieldset.add_field(Field('Kh_meridional', data=10 * np.ones(size2D),
                         lon=fset_currents.U.grid.lon, lat=fset_currents.U.grid.lat,
                         mesh='spherical', allow_time_extrapolation=True))

In [ ]:
pset = ParticleSet.from_line(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=SampleParticle,  # the type of particles (JITParticle/ScipyParticle/Custom)
                             start=[18.3176, -34.1395], 
                             finish=[18.3176, -34.1501],
                             depth = None,
                             time=(datetime(2018, 10, 1)),
                             lonlatdepth_dtype= np.float64,
                             size= 1000)    

In [ ]:
# Hydrodynamic drag kernel
def WaterDrag(particle, fieldset, time):
        (u,v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        Ac = 386.28  # cross-sectional area of plant, must be entered manually and calculated before hand
        mass = 48.65 # total mass of the kelp plant being simulated
        expo = 1
        waterdragU = ((0.5*1027.3*(u*u)*0.82*Ac)*expo)/mass # syntax for calculating the loss in velocity as a result of drag
        waterdragV = ((0.5*1027.3*(v*v)*0.82*Ac)*expo)/mass
        kelpvelU = (u - waterdragU)
        kelpvelV = (v - waterdragV)
        particle.lon  += kelpvelU * particle.dt   # overall velocity minus the loss in the velocity as a result of drag
        particle.lat  += kelpvelV * particle.dt   # overall velocity minus the loss in the velocity as a result of drag
        particle.kelpvelU = kelpvelU
        particle.kelpvelV = kelpvelV

In [ ]:
## cast kernels, select appropriate 'kernel set'

#kernels = pset.Kernel(AdvectionRK4) + SampleDistance + BrownianMotion2D # for passive RK4 simulation
kernels = pset.Kernel(AdvectionRK4) + SampleDistance + SampleKelpVel + BrownianMotion2D + WaterDrag # KelpFloat

In [ ]:
## execute simulation

## define output file, select outfile according to simulation type

#pset.ParticleFile(name="RK4_PFloat.nc", outputdt=timedelta(hours=1)), 
#pset.ParticleFile(name="Max_ShapeFloat_H100_W00.nc", outputdt=timedelta(hours=1)), 

pset.execute(kernels, 
             runtime=timedelta(days=30),
             dt=timedelta(hours=1),
             output_file=pset.ParticleFile(name="Max_ShapeFloat_H100_W00_Oct.nc", outputdt=timedelta(hours=1)), 
             recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
# simple plot for viewing, use for checking simulation output

#plotTrajectoriesFile('Max_KFloat_H100_W00.nc')
plotTrajectoriesFile('Max_ShapeFloat_H100_W00.nc')


In [ ]:
plotTrajectoriesFile('Max_KFloat_H100_W00.nc', mode='movie2d_notebook')  
#plotTrajectoriesFile('Max_ShapeFloat_H100_W00.nc', mode='movie2d_notebook')  

In [ ]:
pset.show(field=fieldset.V, show_time=datetime(2018, 1, 30, 0), domain={'N':-30, 'S':-35, 'E':21, 'W':15}, savefile = 'vector_day30', with_particles = False)

In [ ]:
pset.show(animation = True, field='vector', domain={'N':-30, 'S':-35, 'E':21, 'W':15}, savefile = 'vector_day', with_particles = False)